In [1]:
import sys
import os

SCRIPT_DIR = os.path.dirname(os.path.abspath("..."))
sys.path.append(os.path.dirname(SCRIPT_DIR))
from training.generate import (generate_response, load_model_tokenizer_for_generate, 
                               get_special_token_id, get_special_token_id)
from training.transcript_trainer import PROMPT_FORMAT, create_data_set_from_json_list
from datasets import Dataset, DatasetDict, load_dataset, load_from_disk


In [7]:
import json
json_list_file = "/opt/home/bo_ling/dataset/eats_receipt_gcp_v3_train.jsonl"
def gen():
    with open(json_list_file) as file:
        while True:
            line = file.readline()

            # if line is empty
            # end of file is reached
            if not line:
                break 
            try:
                data = json.loads(line)

            except:
                # print(f"BAD DATA: {line}")
                data = json.loads(line.replace("\\", ""))

            yield{
                "input_text": data['input_text'], 
                "output_text": data['output_text'].replace(" total_price", "; total_price").replace(" total_tax", "; total_tax")
            }
dataset = Dataset.from_generator(gen)
dataset = dataset.train_test_split(test_size=0.01)
dataset.save_to_disk("/opt/home/bo_ling/dataset/eats_receipt_gcp_v3_train.hf")

Found cached dataset generator (/home/prodadmin/.cache/huggingface/datasets/generator/default-1066e70b75cec04f/0.0.0)


Saving the dataset (0/1 shards):   0%|          | 0/1399 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/15 [00:00<?, ? examples/s]

In [8]:
train_data = load_from_disk("/opt/home/bo_ling/dataset/eats_receipt_gcp_v3_train.hf")
train_data

DatasetDict({
    train: Dataset({
        features: ['input_text', 'output_text'],
        num_rows: 1399
    })
    test: Dataset({
        features: ['input_text', 'output_text'],
        num_rows: 15
    })
})

In [5]:
import json
json_list_file = "/opt/home/bo_ling/dataset/eats_receipt_gcp_v3_test.jsonl"
def gen():
    with open(json_list_file) as file:
        while True:
            line = file.readline()

            # if line is empty
            # end of file is reached
            if not line:
                break 
            try:
                data = json.loads(line)

            except:
                # print(f"BAD DATA: {line}")
                data = json.loads(line.replace("\\", ""))

            yield{
                "input_text": data['input_text'], 
                "output_text": data['output_text'].replace(" total_price", "; total_price").replace(" total_tax", "; total_tax")
            }
dataset = Dataset.from_generator(gen)
dataset = dataset.train_test_split(test_size=0.99)
dataset.save_to_disk("/opt/home/bo_ling/dataset/eats_receipt_gcp_v3_test.hf")

Generating train split: 0 examples [00:00, ? examples/s]

Dataset generator downloaded and prepared to /home/prodadmin/.cache/huggingface/datasets/generator/default-d79e4a51579adc11/0.0.0. Subsequent calls will reuse this data.


Saving the dataset (0/1 shards):   0%|          | 0/3 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/350 [00:00<?, ? examples/s]

In [6]:
test_data = load_from_disk("/opt/home/bo_ling/dataset/eats_receipt_gcp_v3_test.hf")
test_data

DatasetDict({
    train: Dataset({
        features: ['input_text', 'output_text'],
        num_rows: 3
    })
    test: Dataset({
        features: ['input_text', 'output_text'],
        num_rows: 350
    })
})